In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=ce443779355d9c166024646f3f207c3b42061cc3811e5deec2ac9e8846b54406
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
Download 100%.


In [ ]:
!signate list

  competitionId  title                                                                             closing     prize        submitters
---------------  --------------------------------------------------------------------------------  ----------  ---------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                              -                              7371
             24  【練習問題】お弁当の需要予測                                                      -                              9036
             27  【練習問題】Jリーグの観客動員数予測                                               -                              2268
            100  【練習問題】手書き文字認識                                                        -           Knowledge           395
            102  【練習問題】タイタニックの生存予測                                                -           Knowledge          3310
            103  【練習問題】音楽ラベリング                                                        -           Knowledge           203
            

In [ ]:
! signate files --competition-id=264

  fileId  name               title                       size  updated_at
--------  -----------------  ----------------------  --------  -------------------
     810  train.csv          学習用データ            24772732  2019-09-26 06:16:33
     811  test.csv           検証用データ            24399728  2019-09-26 06:17:00
     814  sample_submit.csv  応募用サンプルファイル    403585  2019-09-26 06:19:00


In [ ]:
!signate download --competition-id=264

sample_submit.csv

test.csv

train.csv


Download completed.


In [ ]:
import pandas as pd
import numpy as np

# ヘッダ有りとヘッダ無しCSVに区別して読み取る
train = pd.read_csv("/content/train.csv", header=0)
test = pd.read_csv("/content/test.csv")
submit = pd.read_csv("/content/sample_submit.csv", header=None, names=["id", "賃料"])

In [ ]:
# データ結合(番号を振り直す)
combined = pd.concat([train,test],axis=0,ignore_index=True)

In [ ]:
# 編集する特徴量
# (済)所在地 = 東京23区
# (済)アクセス = 路線、最寄り駅、徒歩○分
# (済)間取り = そのまま
# (済)築年数 = ヶ月単位に変換
# (済)方角 = そのまま
# (済)面積 = 数値に治す
# (済)所在階 = 階数に変換
#----------------------------------------休憩
# バス・トイレ = バス・トイレ別フラグ　-> 面倒臭すぎるので後回し
# キッチン = コンロタイプで分類 -> これも面倒くさい
# 放送・通信 = ネット対応、ケーブルテレビ、CS
# 室内整備 = エアコン、シューズボックス
# 駐車場 = 有りorなし
# 周辺環境 = 小学校、大学
# 建築構造 = RC
# 契約期間 = 年数

In [ ]:
combined["放送・通信"].value_counts()

,count
放送・通信,
インターネット対応,5166
光ファイバー,4603
インターネット対応／\t光ファイバー,3122
光ファイバー／\tCATV,2840
インターネット対応／\t光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ,2741
...,...
高速インターネット／\tCATV／\tCSアンテナ／\t有線放送,1
光ファイバー／\t有線放送／\tインターネット使用料無料,1
インターネット対応／\tCATV／\t有線放送／\tインターネット使用料無料,1


In [ ]:
# 所在階 = 階数に変換。欠損は2階を入れておく
combined["所在階"] = combined["所在階"].fillna("2")
combined["階"] = 0
for i in range(len(combined)):
  combined.loc[i, "階"] = re.split("[階]", combined["所在階"][i])[0]

<ipython-input-157-ae9727be921d>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined.loc[i, "階"] = re.split("[階]", combined["所在階"][i])[0]


In [ ]:
# 面積(数値に直す)
combined["面積値"] = 0
for i in range(len(combined)):
  combined.loc[i, "面積値"] = combined["面積"][i][:-2]

combined["面積値"] = combined["面積値"].astype("float32")

<ipython-input-158-646a66fcdb2d>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined.loc[i, "面積値"] = combined["面積"][i][:-2]


In [ ]:
# 築年数(ヶ月に換算する)
import re

combined["築ヶ月"] = 0
for i, val in enumerate(combined["築年数"]):

  # 「新築」表記を無くす
  if(val == "新築"):
    combined.loc[i,"築ヶ月"] = "0"
    total = int(0)

  else:
    year = re.split("[年月ヶ]",combined["築年数"][i])[0]
    month = re.split("[年月ヶ]",combined["築年数"][i])[1]
    total = int(year) * 12 + int(month)
  combined.loc[i,"築ヶ月"] = total

combined["築ヶ月"] = combined["築ヶ月"].astype("int32")

<ipython-input-159-e9c2f2068d50>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined.loc[i,"築ヶ月"] = "0"


In [ ]:
# アクセス = 「路線」「最寄り駅」「徒歩時間」に分ける
import re

combined["路線"] = 0
combined["最寄り駅"] = 0
combined["徒歩時間"] = 0
for row in range(len(combined)):
  combined.loc[row,"路線"] = re.split("[\t]", combined["アクセス"][row])[0]
  combined.loc[row,"最寄り駅"] = re.split("[\t]", combined["アクセス"][row])[1]
  combined.loc[row,"徒歩時間"] = re.split("[\t]", combined["アクセス"][row])[2][-2:-1]

<ipython-input-160-8555a5b0c86d>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '都営三田線' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined.loc[row,"路線"] = re.split("[\t]", combined["アクセス"][row])[0]
<ipython-input-160-8555a5b0c86d>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '西巣鴨駅' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined.loc[row,"最寄り駅"] = re.split("[\t]", combined["アクセス"][row])[1]
<ipython-input-160-8555a5b0c86d>:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined.loc[row,"徒歩時間"] = re.split("[\t]", combined["アクセス"][row])[2][-2:-1]


In [ ]:
# 所在地 = 23区で分ける
import re

combined["地区"] = 0
for row in range(len(combined)):
  combined.loc[row,"地区"] = re.split("[都区]", combined["所在地"][row])[1]

<ipython-input-161-6ffc6b8ef027>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '北' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined.loc[row,"地区"] = re.split("[都区]", combined["所在地"][row])[1]


In [ ]:
# # 間取り = あまり関係ないように見えるので保留
# # (「納戸」を抹消する)
# combined["間取り2"] = 0

# for i in range(len(combined)):
#   if(combined.loc[i,"間取り"][-4:])=="(納戸)":
#     combined.loc[i,"間取り2"] = combined.loc[i,"間取り"][:-6]
#   else:
#     combined.loc[i,"間取り2"] = combined.loc[i,"間取り"]

<ipython-input-187-d5aaae060db0>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1K' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined.loc[i,"間取り2"] = combined.loc[i,"間取り"]


In [ ]:
# 取り除く変数
RMV = ["id", "賃料"]
FEATURES = [c for c in combined.columns if not c in RMV]

# カテゴリ変数の分類
CATS = []
for c in FEATURES:
  if combined[c].dtype == "object":
    CATS.append(c)
    combined[c] = combined[c].fillna("NAN")
    # test[c] = test[c].fillna("NAN")

In [ ]:
CATS

['所在地',
 'アクセス',
 '間取り',
 '築年数',
 '方角',
 '面積',
 '所在階',
 'バス・トイレ',
 'キッチン',
 '放送・通信',
 '室内設備',
 '駐車場',
 '周辺環境',
 '建物構造',
 '契約期間',
 '階',
 '路線',
 '最寄り駅',
 '徒歩時間',
 '地区']

In [ ]:
# 特徴量の整理
for c in FEATURES:

  # カテゴリ変数の場合はラベルエンコード
  for c in CATS:
    combined[c],_ = combined[c].factorize()

    combined[c] -= combined[c].min() # 最小値で引く
    combined[c] = combined[c].astype("int32")

  # 数値の場合はメモリを減らす
  else:
    if combined[c].dtype == "float64":
      combined[c] = combined[c].astype("float32")
    if combined[c].dtype == "int64":
      combined[c] = combined[c].astype("int32")

# 結合を戻す
train = combined.iloc[:len(train)].copy()
test = combined.iloc[len(train):].reset_index(drop=True).copy()

In [ ]:
# ベースラインを作成する
from sklearn.model_selection import KFold # K分割
from xgboost import XGBRegressor, XGBClassifier
import xgboost
import lightgbm as lgb
from lightgbm import LGBMRegressor, LGBMClassifier

# LGBMパラメータ
lgbm_params = {
    "device": "cpu",
    "max_depth": 3,
    # "colsample_bytree": 0.4,
    # "subsample": 0.9,
    "n_estimators": 2500,
    "learning_rate": 0.02,
    "objective": "regression",
    # "boosting": "gbdt",
    "seed": 1234,
}

In [ ]:
%%time
FOLDS = 5 # 分割数
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

# 検証用結果(valie)
oof_lgb = np.zeros(len(train))

# テスト用結果(test)
pred_lgb = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(kf.split(train)):

  print("#"*25)
  print(f"### Fold {i+1}")
  print("#"*25)

  # インデックスに割り振る

  x_train = train.loc[train_index, FEATURES].copy()
  y_train = train.loc[train_index, "賃料"]
  x_valid = train.loc[test_index, FEATURES].copy()
  y_valid = train.loc[test_index, "賃料"]
  x_test = test[FEATURES].copy()

  # データセット作成
  # lgb_train = lgb.Dataset(x_train, y_train)
  # lgb_eval = lgb.Dataset(x_valid, y_valid, reference = lgb_train)

  # 学習
  model_lgb = LGBMRegressor(**lgbm_params,
                            verbose=-1,)

  model_lgb.fit(x_train, y_train,
                eval_set = [(x_valid, y_valid)],
                )

  # 予測
  # y_pred = model_lgb.predict(x_valid, num_iteration = model_lgb.best_iteration)
  oof_lgb[test_index] = model_lgb.predict(x_valid)
  pred_lgb += model_lgb.predict(x_test)

# 平均評価
pred_lgb /= FOLDS

#########################
### Fold 1
#########################
#########################
### Fold 2
#########################
#########################
### Fold 3
#########################
#########################
### Fold 4
#########################
#########################
### Fold 5
#########################
CPU times: user 1min 15s, sys: 203 ms, total: 1min 15s
Wall time: 19 s


In [ ]:
from sklearn.metrics import mean_squared_error

# 検証用スコアデータの算出
y_true = train["賃料"].tolist()
m = np.sqrt(mean_squared_error(y_true, oof_lgb.tolist()))
print(f"\nOverall CV for LightGBM = ",m)


Overall CV for LightGBM =  25649.921765209732


In [ ]:
submit["賃料"] = pd.DataFrame(pred_lgb)

In [ ]:
submit.to_csv("submit.csv", header=False, index=False)